In [ ]:
import math

def lerp(a, b, p):
    return a + (b - a) * p

def ignition_time(a, b, c, t):
    # a - Gravitational acceration
    # b - Current velocity
    # c - Current altitude
    # t - Acceleration on powered descent

    # The discriminant
    D = b**2 - 4*a*c

    # New a
    na = a + t

    # The time (in seconds) until iginition for optimal landing.
    k = -(math.sqrt(t * na * D) + b * t) / (2 * a * t)

    # The time until touchdown if ignition occures at k
    x = (2*t*k - b) / (2 * na)

    return k, x


def get_positive_area(b, v, c):
    # 2a
    _2a = v-b

    # a
    a = _2a/2

    # Discriminant 
    D = b**2 - 4 * a * c

    # Square root of the discriminant if it's not imaginary and if it is, replace it with 0.
    D_ = math.sqrt(D) if D >= 0 else 0

    # The area of the rectangle with 2 edges lying on the positive y and negative x axis and the top edge at y = c where c is the intersection point of the paraboly with the y axis, and the left edge at the x = x₀ where x₀ is the center of the parabola.
    cx0 = c * -b / _2a

    return (D_**3 + b**3) / (12*a**2) + cx0

In [ ]:
class PID:
    def __init__(self, k_p, k_i, k_d):
        self.p = 0
        self.i = 0
        self.d = 0

        self.k_p = k_p
        self.k_i = k_i
        self.k_d = k_d

        self.p_err = None

    def __call__(self, err, dt=1):
        if self.p_err is not None:
            self.p = self.k_p * err  # Proportional
            self.i += self.k_i * err * dt # Integral
            self.d = self.k_d * (err - self.p_err) / dt # Derivative

        self.p_err = err

        return self.p + self.i + self.d

    def get_pid(self):
        return self.p, self.i, self.d

In [ ]:
import math
import numpy as np
import pandas as pd


class Rocket:
    def __init__(self, dry_mass, alt, fuel, thrust, fuel_consumption, vel, state,
                 target_alt, time_steps, gravity=9.8):
        self.dry_mass = dry_mass
        self.alt = alt
        self.fuel = fuel
        self.thrust = thrust
        self.vel = vel
        self.acc = 0
        self.throttle = 0
        self.fuel_consumption = fuel_consumption
        self.state = state
        self.p_state = state
        self.state_n = 0
        self.target_alt = target_alt
        self.pid = PID(0.25, 0.0005, 1.8)
        self.time_steps = time_steps
        self.dt = 1 / time_steps
        self.pid(self.target_alt - self.alt, self.dt)
        self.g = gravity
        self.time = 0

        self.k = 0
        self.estimated_touchdown = 0

        # Angular attributes
        self.angle_x = 0  # Current angle of the rocket in the x direction (in degrees)
        self.angular_velocity_x = 0  # Angular velocity of the rocket in the x direction
        self.target_angle_x = 0  # Desired angle in the x direction (usually 0 for vertical flight)
        self.angle_pid_x = PID(1.5, 0.1, 0.7)  # PID controller for angle in x direction

        self.angle_y = 0  # Current angle of the rocket in the y direction (in degrees)
        self.angular_velocity_y = 0  # Angular velocity of the rocket in the y direction
        self.target_angle_y = 0  # Desired angle in the y direction (usually 0 for vertical flight)
        self.angle_pid_y = PID(1.5, 0.1, 0.7)  # PID controller for angle in y direction

        self.data_keys = [
            'alt', 'fuel', 'vel', 'acc', 'throttle', 'ed', 't/w', 'state_n', 'state', 'k',
            'desired_throttle', 'pid_p', 'pid_i', 'pid_d', 'desired_acceleration', 'ΔV', 'estimated touchdown',
            'angle_x', 'angular_velocity_x', 'desired_angle_x', 'angle_pid_p_x', 'angle_pid_i_x', 'angle_pid_d_x',
            'angle_y', 'angular_velocity_y', 'desired_angle_y', 'angle_pid_p_y', 'angle_pid_i_y', 'angle_pid_d_y'
        ]

    def update(self):
        self.time += self.dt
        self._physics()
        self._guidance()

        # Wind perturbations in both directions
        wind_perturbation_x = np.random.normal(0, 5)
        wind_perturbation_y = np.random.normal(0, 5)
        self.angle_x += wind_perturbation_x
        self.angle_y += wind_perturbation_y

        if self.state != self.p_state:
            self.state_n += 1
            self.p_state = self.state

        return {
            'alt': self.alt,
            'vel': self.vel,
            'acc': self.acc,
            'throttle': self.throttle,
            'fuel': self.fuel,
            'ed': self.estimated_distance(),
            'desired_throttle': self.desired_throttle,
            'desired_acceleration': self.desired_acc,
            'pid_p': self.pid.p,
            'pid_i': self.pid.i,
            'pid_d': self.pid.d,
            't/w': self.thrust / (self.dry_mass + self.fuel) / self.g,
            'state_n': self.state_n,
            'state': self.state,
            'ΔV': self.ΔV,
            'k': self.k,
            'estimated touchdown': self.estimated_touchdown,
            'angle_x': self.angle_x,
            'angular_velocity_x': self.angular_velocity_x,
            'desired_angle_x': self.target_angle_x,
            'angle_pid_p_x': self.angle_pid_x.p,
            'angle_pid_i_x': self.angle_pid_x.i,
            'angle_pid_d_x': self.angle_pid_x.d,
            'angle_y': self.angle_y,
            'angular_velocity_y': self.angular_velocity_y,
            'desired_angle_y': self.target_angle_y,
            'angle_pid_p_y': self.angle_pid_y.p,
            'angle_pid_i_y': self.angle_pid_y.i,
            'angle_pid_d_y': self.angle_pid_y.d
        }

    def _physics(self):
        if self.alt <= 0:
            self.alt = 0
            self.vel = 0
            self.acc = 0
            return True

        self.throttle = self.throttle * min(self.fuel / self.fuel_consumption, 1)

        self.acc = self.throttle * self.thrust / (self.dry_mass + self.fuel) - self.g

        self.vel += self.acc * self.dt
        self.alt += self.vel * self.dt

        if self.fuel * self.dt >= self.throttle:
            self.fuel -= self.throttle * self.fuel_consumption * self.dt
            if self.fuel < 0:
                self.fuel = 0

        self.angular_velocity_x += self.angle_pid_x(self.target_angle_x - self.angle_x, self.dt)
        self.angle_x += self.angular_velocity_x * self.dt

        self.angular_velocity_y += self.angle_pid_y(self.target_angle_y - self.angle_y, self.dt)
        self.angle_y += self.angular_velocity_y * self.dt

    def _guidance(self):
        self.desired_acc = self.pid(self.target_alt - self.alt, self.dt) + self.g

        if self.state == 'landing-0':
            if self.alt < 20:  # Start earlier
                self.desired_throttle = 0.6  # Start with a lower throttle
                self.state = 'landing-1'
            else:
                self.desired_throttle = 0

        if self.state == 'landing-1':
            if self.vel < 0:
                self.desired_throttle = self.desired_acc / (self.thrust / (self.dry_mass + self.fuel)) - 0.2  # Linear throttle adjustment with higher initial value
                self.desired_throttle = max(min(self.desired_throttle, 1), 0)  # Clamp between 0 and 1
            else:
                self.desired_throttle = 0
                self.state = 'landed'

        elif self.state == 'hover':
            self.desired_throttle = self.desired_acc / (self.thrust / (self.dry_mass + self.fuel))

        self.throttle = min(max(0, self.desired_throttle), 1)

    @property
    def ΔV(self):
        self.exhaust_velocity = self.thrust / self.fuel_consumption
        return self.exhaust_velocity * math.log((self.dry_mass + self.fuel) / self.dry_mass)

    def estimated_distance(self):
        x0 = self.g - self.thrust * .95 / (self.dry_mass + self.fuel - self.fuel_consumption * 0.0)
        x1 = self.g - self.thrust * .95 / (self.dry_mass + self.fuel - self.fuel_consumption * 1.0)
        u = x0
        v = x1
        y = abs(self.vel)
        return get_positive_area(u, v, y)

    def should_fire(self):
        a = -self.g
        b = self.vel
        c = self.alt
        t = self.thrust * 0.95 / (self.dry_mass + self.fuel)
        k, x = ignition_time(a, b, c, t)
        self.k = k
        self.estimated_touchdown = x
        if int(self.time * 100) % 100 == 0:
            print(f'Time: {int(self.time)}\n a: {a} b: {b} c: {c} t: {t}\n k: {k} x: {x}')
        return k <= 1


def run():
    simulation_duration_s = 120
    simulation_timesteps = 50
    data_frequency_p = 10

    data = {}
    r = Rocket(dry_mass=100,
               alt=1,
               fuel=450,
               thrust=6500,
               fuel_consumption=10,
               vel=0,
               state='hover',
               target_alt=20,
               time_steps=simulation_timesteps)

    for key in r.data_keys:
        data[key] = []

    for i in range(0, simulation_duration_s * simulation_timesteps):
        for k, v in r.update().items():
            if i % data_frequency_p == 0:
                data[k].append(v)

        if i == simulation_duration_s * simulation_timesteps / 4:
            r.state = 'landing-0'

        if i > data_frequency_p * 20:
            if data['alt'][-10] <= 0:
                break

    df = pd.DataFrame(data)
    df.to_csv('/tmp/rocket_simulation_data.csv', index=False)
    return data


run()



In [ ]:
import os
import sys
import time

import matplotlib
matplotlib.use('Agg')
import numpy as np
import matplotlib.pyplot as plt

def main():
    results = run()
    zero_line = [0 for a in results["alt"]]
    
    plt.style.use('Solarize_Light2')

    fig = plt.figure(figsize=(18, 18))
    axs = [0] * 14 
    axs[0] = fig.add_subplot(4, 2, 1)
    axs[1] = fig.add_subplot(4, 4, 5)
    axs[2] = fig.add_subplot(8, 4, 19)
    axs[3] = fig.add_subplot(8, 4, 23)
    axs[4] = fig.add_subplot(4, 2, 5)
    axs[5] = fig.add_subplot(8, 4, 10)
    axs[6] = fig.add_subplot(8, 4, 11)
    axs[7] = fig.add_subplot(8, 4, 12)
    axs[8] = fig.add_subplot(8, 4, 15)
    axs[9] = fig.add_subplot(8, 4, 16)
    axs[10] = fig.add_subplot(4, 2, 2)
    axs[11] = fig.add_subplot(8, 4, 14)
    axs[12] = fig.add_subplot(4, 4, 12)
    axs[13] = fig.add_subplot(4, 4, 13)

    axs[0].plot(results["alt"])
    axs[0].plot([a-e if s[:7] == "landing" else a for a,e,s in zip(results["alt"], results["ed"], results["state"])], '--', alpha=0.3)
    for ax in axs:
        for i in range(len(set(results["state_n"]))-1):
            ax.axvspan(results["state_n"].index(i), results["state_n"].index(i+1), color=['c', 'r', 'g', 'y'][i], alpha=0.1, lw=0)
    axs[0].set_title('altitude')

    axs[1].plot(results["vel"])
    axs[1].set_title('velocity')

    axs[2].plot(results["fuel"])
    axs[2].set_ylim([-10, np.max(results["fuel"])+10])
    axs[2].set_title('fuel')

    axs[3].plot(results["ed"])
    axs[3].set_title('estimated distance')

    axs[4].plot(results["acc"])
    axs[4].plot(zero_line, alpha=0.5)
    axs[4].set_title('acceleration')

    axs[5].plot(results["throttle"])
    axs[5].plot(results["desired_throttle"], '--', color='y', alpha=1, linewidth=2)
    axs[5].set_ylim([-0.1, 1.1])
    axs[5].set_title('throttle')

    axs[11].plot(results["throttle"])
    axs[11].set_ylim([-0.1, 1.1])
    axs[11].set_xlim([results["state"].index('landing-1')-5, results["state"].index('landed')+5])
    axs[11].set_title('throttle - powered descent')

    axs[6].plot(results["pid_p"])
    axs[6].set_title('pid_p')

    axs[7].plot(results["pid_i"])
    axs[7].set_title('pid_i')

    axs[8].plot(results["pid_d"])
    axs[8].set_title('pid_d')

    axs[9].plot(results["desired_acceleration"], '--')
    axs[9].plot(results["pid_p"], alpha=0.4)
    axs[9].plot(results["pid_i"], alpha=0.4)
    axs[9].plot(results["pid_d"], alpha=0.4)
    axs[9].set_title('pid all')

    axs[10].plot(results["t/w"])
    axs[10].set_title('thrust / weight')

    axs[12].plot(results['ΔV'])
    axs[12].set_title('ΔV left')

    # New plot for angle control
    axs[13].plot(results["angle"])
    axs[13].set_title('angle')

    fig.tight_layout()

    # Save figure to Downloads folder
    downloads_directory = os.path.join(os.path.expanduser('~'), 'Downloads')
    file_name = f'flight_simulation.png'
    file_path = os.path.join(downloads_directory, file_name)

    if not os.path.exists(downloads_directory):
        os.makedirs(downloads_directory)

    plt.savefig(file_path)

if __name__ == '__main__':
    main()

